In [169]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math

### 表示系関数

In [170]:
# 分を24時間表示に変換
def calculate_time(minutes):
    # 9:00 を0分、20:00 を660分として計算
    start_time = 9 * 60  # 9:00を分単位に変換
    end_time = 20 * 60  # 20:00を分単位に変換
    
    total_minutes = start_time + minutes
    
    # 時間と分に変換
    hours = total_minutes // 60
    mins = total_minutes % 60
    
    # 24時間表記に変換
    if hours >= 24:
        hours -= 24
    
    # 時間と分を文字列に変換して返す
    return f"{hours:02d}:{mins:02d}"

In [171]:
# 24時間表示を分に変換
def calculate_minutes(time_str):
    # 時刻を時間と分に分割
    hours, mins = map(int, time_str.split(":"))
    
    # 9:00 を0分、20:00 を660分として計算
    start_time = 9 * 60  # 9:00を分単位に変換
    end_time = 20 * 60  # 20:00を分単位に変換
    
    # 入力された時刻を分単位に変換
    total_minutes = hours * 60 + mins
    
    # 24時間以上の場合は調整
    if total_minutes < start_time:
        total_minutes += 24 * 60
    
    # 開始時刻からの経過時間を計算
    elapsed_minutes = total_minutes - start_time
    
    return elapsed_minutes

In [172]:
def not_in_new_ride(a, b):
    ans=a
    # リストaからリストbに含まれている要素を取り除く
    for item in b:
        if item in a:
            ans.remove(item)
    return ans

In [185]:
def ride_printer(date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time):
    print("-" * 40)
    print(f"~ {date} のデータによる乗車プラン ~")
    print("乗車時間　：　アトラクション名")
    for i in range(len(new_ride_lst)):
        print(f"{calculate_time(ride_time[i])}　： {attractions.get(new_ride_lst[i])}")
    print('トータル所要時間：',total_time)
    print('トータル満足度　：',total_popularity)
    print("-" * 40)
    if not(len(ride_lst)==len(new_ride_lst)):
        print("※全てのアトラクションを乗ることができません")
        print("プランから除外されたアトラクション")
        not_ride = not_in_new_ride(ride_lst, new_ride_lst)
        for i in not_ride:
            print(f"{i}　： {attractions.get(i)}")

### 計算系関数

In [174]:
# 行数を指定すると所用時間を返す関数
def knapsack(row_x, data_x,required_time_x, waiting_time_x, move_time_x, attraction_no_x):
    waiting_time_x = data_x.iloc[row_x, attraction_no_x]
    ans = required_time_x + waiting_time_x + move_time_x
    return ans

In [175]:
# 合計所要時間と合計人気度を算出する関数
def ride_calculation(data, ride_lst, required_times, move_time):
    total_popularity = 0
    total_time = 0
    ride_time = []
    new_ride_lst = []
    # start_time = data.index[0]
    # print("start time is:", calculate_time(start_time))
    row = 0
    try:
        for i in ride_lst:
            # total_timeが675以上かつiが0または1の場合にループを強制終了
            # ソアリン、トイストーリーMは20:15以降DPA所持者のみ乗車可能
            if (data.index[row]) >= 675 and (i == 0 or i == 1):
                continue
            
            else:
                # iが8以上の場合、次のループへ(該当しないアトラクション)
                if i >= 8 or i < 0:
                    # print("不明なアトラクション：", i)
                    continue
                else:
                    # print("-" * 40)
                    # print(calculate_time(data.index[row]))
                    ride_time.append(data.index[row])
                    # print(attractions.get(i))
                    # print("移動時間", move_time)
                    waiting_time = data.iloc[row, i]
                    # print("待ち時間:", waiting_time)
                    required_time = required_times[i]
                    # print("観賞時間:", required_time)
                    ans = knapsack(row, data, required_time, waiting_time, move_time, i)
                    # print("合計所要時間:", ans)
                    total_time += ans
                    row = int(np.ceil(total_time / 15))
                    total_popularity += popularity[i]
                    new_ride_lst.append(i)
    except Exception as e:
        #print("全てのアトラクションを乗ることができません")
        pass

    # print("-" * 40)
    return total_popularity, total_time, new_ride_lst, ride_time

### optuna

In [188]:
import optuna
import pandas as pd
import re

attractions = {
    0: 'ソアリン',
    1: 'トイストーリーマニア',
    2: 'タワーオブテラー',
    3: 'センターオブジアース',
    4: 'インディージョーンズ',
    5: 'レイジングスピリッツ',
    6: 'マジックランプシアター',
    7: 'タートルトーク'
}


best_plan_lst = []

# 目的関数の定義
def objective(trial):
    # アトラクションの所要時間と人気度
    required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
    popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
    move_time = 10
    
    file_path = "date_data2/2022-07-01.csv"
    data = pd.read_csv(file_path)
    data = data.set_index('時間')
    
    # 乗り物の数を決定
    n_rides = trial.suggest_int('n_rides', 1, 15)  # 最大20つの乗り物を仮定
    
    # 乗り物の定義
    ride_lst = [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]
    
    # ride_calculation関数の呼び出し
    total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, move_time)
    
    # 制約条件の範囲
    max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
    
    # 制約を満たす場合のみ最適化対象とする
    if total_time <= max_allowed_time:
        global best_plan
        
        # 正規表現で年月日部分を抽出
        match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
        date = match.group() if match else None
        
        best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
        return total_popularity
    else:
        # 制約を満たさない場合はペナルティを課す
        return float('-inf')

import time
start_time = time.time()


# Optunaによる最適化
study = optuna.create_study(direction="maximize")  # 人気度を最大化
study.optimize(objective, n_trials=1000)

# 結果の表示
best_trial = study.best_trial
print(f"Best trial: {best_trial.number}")
print(f"Best popularity: {best_trial.value}")
print(f"Parameters: {best_trial.params}")

end_time = time.time()

# 処理時間を計算
elapsed_time = end_time - start_time

# 処理時間を表示
print(f"処理時間: {elapsed_time} 秒")

[I 2024-07-04 11:20:27,231] A new study created in memory with name: no-name-ac05184f-750e-457d-a7d0-2d892195a115
[I 2024-07-04 11:20:27,243] Trial 0 finished with value: 133.0 and parameters: {'n_rides': 3, 'ride_0': 3, 'ride_1': 4, 'ride_2': 2}. Best is trial 0 with value: 133.0.
[I 2024-07-04 11:20:27,250] Trial 1 finished with value: 145.0 and parameters: {'n_rides': 3, 'ride_0': 3, 'ride_1': 3, 'ride_2': 2}. Best is trial 1 with value: 145.0.
[I 2024-07-04 11:20:27,256] Trial 2 finished with value: 407.0 and parameters: {'n_rides': 9, 'ride_0': 4, 'ride_1': 6, 'ride_2': 1, 'ride_3': 2, 'ride_4': 6, 'ride_5': 0, 'ride_6': 2, 'ride_7': 5, 'ride_8': 3}. Best is trial 2 with value: 407.0.
[I 2024-07-04 11:20:27,261] Trial 3 finished with value: 316.0 and parameters: {'n_rides': 7, 'ride_0': 7, 'ride_1': 0, 'ride_2': 7, 'ride_3': 6, 'ride_4': 3, 'ride_5': 1, 'ride_6': 2}. Best is trial 2 with value: 407.0.
[I 2024-07-04 11:20:27,267] Trial 4 finished with value: 363.0 and parameters: {

Best trial: 730
Best popularity: 883.0
Parameters: {'n_rides': 15, 'ride_0': 4, 'ride_1': 7, 'ride_2': 5, 'ride_3': 2, 'ride_4': 0, 'ride_5': 5, 'ride_6': 1, 'ride_7': 0, 'ride_8': 3, 'ride_9': 0, 'ride_10': 3, 'ride_11': 0, 'ride_12': 1, 'ride_13': 1, 'ride_14': 0}
処理時間: 105.53542399406433 秒


In [186]:
# -------ここからベストプランの表示-------
# DataFrameに変換
best_plan_df = pd.DataFrame(best_plan_lst)

# best_trial.numberに対応する行を抽出
best_plan = best_plan_df[best_plan_df[0] == best_trial.number].iloc[0]

display(best_plan)

# 各要素を取得
date = best_plan[1]
total_popularity = best_plan[2]
total_time = best_plan[3]
ride_lst = best_plan[4]
new_ride_lst = best_plan[5]
ride_time = best_plan[6]

# ride_printerで表示
ride_printer(date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time)

0           117
1    2022-07-01
2            66
3            41
4        [4, 4]
5        [4, 4]
6       [0, 30]
Name: 53, dtype: object

----------------------------------------
~ 2022-07-01 のデータによる乗車プラン ~
乗車時間　：　アトラクション名
09:00　： インディージョーンズ
09:30　： インディージョーンズ
トータル所要時間： 41
トータル満足度　： 66
----------------------------------------
